In [27]:
import torch
import torch.nn as nn
from layer.component import EncoderBlock, DecoderBlock

from PIL import Image
import numpy as np
import torchvision
from torchvision import transforms

In [11]:
class UnetModel(nn.Module):

    def __init__(self, in_channels, out_channels, model_depth=4, final_activation="sigmoid"):
        super(UnetModel, self).__init__()
        self.encoder = EncoderBlock(in_channels=in_channels, model_depth=model_depth)
        self.decoder = DecoderBlock(out_channels=out_channels, model_depth=model_depth)
        if final_activation == "sigmoid":
            self.sigmoid = nn.Sigmoid()
        else:
            self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x, downsampling_features = self.encoder(x)
        x = self.decoder(x, downsampling_features)
        x = self.sigmoid(x)
        print("Final output shape: ", x.shape)
        return x


In [12]:
loss_function = torch.nn.MSELoss(reduction='sum')

In [13]:
device_id = f"cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_id)

In [33]:
batch_size = (5,)
input_shape = (10,10,10,3)
output_shape = (10,10,10,3)

x = torch.randn(batch_size + input_shape)
y = torch.randn(batch_size +output_shape)


trans = transforms.Compose([
    transforms.Resize((10,10)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

# trainset = torchvision.datasets.ImageFolder(root=r'image', transform=trans)

image = Image.open('image/T1W.png')
image.size
# trainset.__getitem__(0)[0].shape


(320, 260)

In [24]:

model = UnetModel(in_channels=10, out_channels=10)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)


for epoch in range(2):
    # 순전파 단계 : 모델에서 x에 대한 예측 값 y_pred를 계산합니다.
    y_pred = model(x)

    loss = loss_function(y_pred,y)

    print(f"epoch %0d{epoch} | lose : {loss}")

    optimizer.zero_grad()

    # 역전파 단계
    loss.backward()

    # optimizer의 step 함수를 호출 하면 매개 변수가 갱신됨
    optimizer.step()

conv_0_0 torch.Size([5, 32, 10, 10, 3])
conv_0_1 torch.Size([5, 64, 10, 10, 3])
max_pooling_0 torch.Size([5, 64, 5, 5, 1])
conv_1_0 torch.Size([5, 64, 5, 5, 1])
conv_1_1 torch.Size([5, 128, 5, 5, 1])


RuntimeError: Given input size: (128x5x5x1). Calculated output size: (128x2x2x0). Output size is too small